# Please run the cells by pressing Shift + Enter for each cell. 
First import the necesseary libraries and the napari viewer.

In [4]:
import numpy as np
import vigra
import napari
import matplotlib
import matplotlib.pyplot as plt
import pickle
import cv2 as cv
import geopandas as gpd
import pandas
import os
import json
import h5py
import tifffile
from sklearn.ensemble import RandomForestClassifier
from skimage.measure import label, regionprops
from skimage import filters, morphology, transform as transf
from scipy import ndimage as ndi, signal
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from shapely.geometry import Polygon, MultiPolygon
from shapely.affinity import translate, scale, affine_transform

In [5]:
# Set up the GUI and start the napari viewer
%matplotlib qt
%gui qt
viewer = napari.Viewer()

# 1. Import geophysical image 
Import the geophysical image to be analysed into the napari viewer using 'File -> Open File(s)', and then run the cell below. If you would like to open a 3D dataset consisting of several horizontal slices, create a 3D file containing the entire volume (e.g. a 3D Tiff) or use the option 'File -> Open Files as Stack'. This will allow you to select all horizontal slice images at once, and import the complete volume in napari. 3D data can be visualised as horizontal slices, vertical slices (xz and yz plane) and as a volume; use the second and third button in the lower left corner of the viewer.<br> Run the code below, to convert the input data to a numpy array, extract the path of the input data and determine the number of dimensions for late use.

In [6]:
imageOrig = viewer.layers[0].data
image = np.array(imageOrig)
path = viewer.layers[0].source.path
nrofdim = np.ndim(imageOrig)

# 2. Machine learning based pixel classification
Pixel classification using the code below will usually be slower than in ilastik or other segmentation tools such as LABKIT or Trainable Weka Segmentation. For 3D data in particular, run times may be significantly longer and RAM usage may be significantly larger. If pixel segmentation has been performed by means of an external tool such as ilastik, please import the segmented image in step 3.1 below.

## 2.1. Load features
If no pixel features have been calculated for your input data, please go to step 2.2.<br>
If pixel features, feature names and scales for the input data have previously been saved, they can be loaded here. Please adjust the file name in the first line of each cell if necessary. The files should be in the same folder as the Jupyter Notebook, or their path should be specified in the code below. After loading the features, please go immediately to step 2.4.

In [13]:
f = open('Features.pckl', 'rb')
features = pickle.load(f)
f.close()
X = features.T

In [8]:
f = open('Feature_names.pckl', 'rb')
feature_names = pickle.load(f)
f.close()

In [9]:
f = open('Feature_scales.pckl', 'rb')
feature_scales = pickle.load(f)
f.close()

## 2.2. Select scales and features
The scales below will be used for the pixel classification. These are appropriate for most geophysical datasets. You can add or remove scales by modifying the code below. If you use a pretrained classifier, please use the same scales and features as used for training the classifier.<br> If an error message appears when calculating the features saying that 'the kernel is longer than the line', this may indicate that one or more scales are too large for the data. This can happen for example in the case of 3D data, when the kernel (which is then also 3D) becomes too large in comparison with the number of horizontal slices. In that case, please remove the largest scale(s).

In [5]:
sigmas = [0.7, 1.0, 1.6, 3.5, 5.0, 10.0]

The image features below will be used for the pixel classification. These are appropriate for most geophysical datasets. You can deselect features by deleting them or commenting them out by putting a '#' at the beginning of the line. Other features can be added and appended to the feature stack.

In [6]:
imageFloat = image.astype('float64')
feature_stack = []
feature_names = []
feature_scales = []

# Gaussian smoothing with sigma = 0.3
gaussian_smoothing = vigra.filters.gaussianSmoothing(image, 0.3)
feature_stack.append(gaussian_smoothing.ravel())
feature_names.append('Gaussian smoothing')
feature_scales.append(0.3)

for sigma in sigmas:   
    # Gaussian smoothing
    gaussian_smoothing = vigra.filters.gaussianSmoothing(imageFloat, sigma)
    feature_stack.append(gaussian_smoothing.ravel()) 
    feature_names.append('Gaussian smoothing')
    feature_scales.append(sigma)

    # Laplacian of Gaussian
    laplacian_of_gaussian = vigra.filters.laplacianOfGaussian(imageFloat, sigma)
    feature_stack.append(laplacian_of_gaussian.ravel()) 
    feature_names.append('Laplacian of Gaussian')
    feature_scales.append(sigma)

    # Gaussian Gradient Magnitude
    gaussian_gradient_magnitude = vigra.filters.gaussianGradientMagnitude(imageFloat, sigma)
    feature_stack.append(gaussian_gradient_magnitude.ravel()) 
    feature_names.append('Gaussian gradient magnitude')
    feature_scales.append(sigma)

    # Difference of Gaussians
    difference_of_gaussians = vigra.filters.gaussianSmoothing(imageFloat, sigma) - vigra.filters.gaussianSmoothing(imageFloat, sigma * 0.66)
    feature_stack.append(difference_of_gaussians.ravel())
    feature_names.append('Difference of Gaussians')
    feature_scales.append(sigma)

    # Stucture tensor eigenvalues
    if nrofdim == 2:
        structure_tensor_eigenvalues = vigra.filters.structureTensorEigenvalues(imageFloat, sigma, sigma * 0.5)[:,:,0]
    else:
        structure_tensor_eigenvalues = vigra.filters.structureTensorEigenvalues(imageFloat, sigma, sigma * 0.5)[:,:,:,0]
    feature_stack.append(structure_tensor_eigenvalues.ravel())
    feature_names.append('Structure tensor eigenvalues')
    feature_scales.append(sigma)

    #Hessian of Gaussian eigenvalues
    if nrofdim == 2:
        hessian_of_gaussian_eigenvalues = vigra.filters.hessianOfGaussianEigenvalues(imageFloat, sigma)[:,:,1]
    else:
        hessian_of_gaussian_eigenvalues = vigra.filters.hessianOfGaussianEigenvalues(imageFloat, sigma)[:,:,:,1]
    feature_stack.append(hessian_of_gaussian_eigenvalues.ravel())    
    feature_names.append('Hessian of Gaussian eigenvalues')
    feature_scales.append(sigma)
features = np.asarray(feature_stack)
X = features.T

## 2.3. Save features
If further classification tests will be carried out on the same input data in the future, it may be useful to store pixel features,  feature names and scales for later use. Please adjust the file names or path names in the first line of each cell if necessary (in the code below, files are saved in the same folder as the Jupyter Notebook). 

In [7]:
f = open('Features.pckl', 'wb')
pickle.dump(features, f)
f.close()

In [8]:
f = open('Feature_names.pckl', 'wb')
pickle.dump(feature_names, f)
f.close()

In [9]:
f = open('Feature_scales.pckl', 'wb')
pickle.dump(feature_scales, f)
f.close()

## 2.4. View features

You can create an overview table of the features and scales by running the code below.

In [10]:
df = pandas.DataFrame(list(zip(feature_names, feature_scales)), columns = ['Feature name', 'Scale'])
print(df)

                       Feature name  Scale
0                Gaussian smoothing    0.3
1                Gaussian smoothing    0.7
2             Laplacian of Gaussian    0.7
3       Gaussian gradient magnitude    0.7
4           Difference of Gaussians    0.7
5      Structure tensor eigenvalues    0.7
6   Hessian of Gaussian eigenvalues    0.7
7                Gaussian smoothing    1.0
8             Laplacian of Gaussian    1.0
9       Gaussian gradient magnitude    1.0
10          Difference of Gaussians    1.0
11     Structure tensor eigenvalues    1.0
12  Hessian of Gaussian eigenvalues    1.0
13               Gaussian smoothing    1.6
14            Laplacian of Gaussian    1.6
15      Gaussian gradient magnitude    1.6
16          Difference of Gaussians    1.6
17     Structure tensor eigenvalues    1.6
18  Hessian of Gaussian eigenvalues    1.6
19               Gaussian smoothing    3.5
20            Laplacian of Gaussian    3.5
21      Gaussian gradient magnitude    3.5
22         

You can view a selected feature by entering its number between the brackets in the first line of the cell below. The number can be found by looking up the feature name and scale in the table above. If the data is 3D, please also enter the horizontal slice to be viewed in the cell below. The image with the selected feature will open in a separate window.

In [14]:
selectedFeature = features[20,:] # Enter number of feature to be viewed
if nrofdim == 2:
    selectedFeatureReshaped = np.reshape(selectedFeature,image.shape)
else:
    selectedSlice = 15 # For 3D data, enter slice to be viewed
    selectedFeatureSlice = selectedFeature[(selectedSlice * image.shape[1] * image.shape[2]):((selectedSlice + 1) * image.shape[1] * image.shape[2])]
    selectedFeatureReshaped = np.reshape(selectedFeatureSlice,image.shape[1:3])
selectedFeatureClip = np.clip(selectedFeatureReshaped,np.mean(selectedFeatureReshaped) - 3 * np.std(selectedFeatureReshaped),np.mean(selectedFeatureReshaped) + 3 * np.std(selectedFeatureReshaped))
plt.imshow(selectedFeatureClip,cmap='gray')

## 2.5. Load classifier
If a classifier has been previously trained and saved, it can be loaded here. Please specify its name in the code below. It should be in the same folder as the Jupyter Notebook, or its path should be specified in the code below. After loading the classifier, please go to step 2.6.2 to apply it to the input data.

In [15]:
f = open('Pixel_classifier.pckl', 'rb')
classifier = pickle.load(f)
f.close()

## 2.6. Iterative training and segmentation
### 2.6.1. Training
If no classifier has been loaded, please first run the code below if a file with labels (in H5 format) is available (for example if the data have been labeled in other software such as ilastik). Please adjust the file name in the first line if necessary. The file should be in the same folder as the Jupyter Notebook, or its path should be specified in the code below. 

In [47]:
filename = "Labels.h5"
with h5py.File(filename, "r") as f:    
    key = list(f.keys())[0]     
    if nrofdim == 2:
        training_labels = f[key][:,:,0]
    else:
        training_labels = f[key][:,:,:,0]     
    viewer.add_labels(training_labels)

If no file with labels is available, please add the 'training_labels' layer to napari by running the cell below.

In [29]:
training_labels = np.zeros(image.shape[:3], dtype=np.uint8)
viewer.add_labels(training_labels)

<Labels layer 'training_labels' at 0x1b980952d60>

For the iterative training and classification, please follow the procedure below.
1. Unless labels have been imported (see the first cell of section 2.6.1 above), train the classifier by selecting the 'training_labels' layer in napari and adding labels to it, using the paint brush and other instruments (see the icons in the top left corner of the viewer under 'Layer controls'). This notebook requires two classes at the stage of pixel classification: background (= label 1) and archaeological structures (= label 2). Further classification occurs at the object level (see Section 3). Nevertheless, in theory it is possible to add more classes at this stage.  
2. When the labels have been added, run the cells below to train the classifier and carry out the segmentation (step 2.6.2). After the segmentation has been completed, the result of the segmentation is automatically added as the 'segmentation' layer in napari. 
3. Additional labels can then be added in napari (in the 'training_labels' layer), and training and segmentation can be carried out again by running the steps below, in an iterative way, until the result of the segmentation is satisfactory.
4. The labels can be saved by selecting the 'training_labels' layer and applying 'File -> Save Selected Layer(s)' in napari.

In [33]:
if 'segmentation' in viewer.layers: 
    viewer.layers.remove('segmentation')
training_labels = viewer.layers['training_labels'].data

In [34]:
y = training_labels.ravel()
mask = y > 0
Xsel = X[mask]
ysel = y[mask]
classifier = RandomForestClassifier(n_jobs=-1)
classifier.fit(Xsel, ysel)

RandomForestClassifier(n_jobs=-1)

### 2.6.2. Segmentation
Please run the code below to apply the trained classifier to the input data. The results of the segmentation are added as the 'segmentation' layer in napari.

In [16]:
result = classifier.predict(X)
segmentation = result.reshape(image.shape)
viewer.add_labels(segmentation, opacity = 0.4)

<Labels layer 'segmentation' at 0x2751b8b4cd0>

## 2.7. Save classifier
To save the classifier, please specify its name below. In the default code below, it is saved in the same folder as the Jupyter Notebook. Otherwise, its path should be specified. 

In [38]:
f = open('Pixel_classifier.pckl', 'wb')
pickle.dump(classifier, f)
f.close()

## 2.8. Apply hysteresis threshold and label the resulting binary image
### 2.8.1. Calculation of probabilities

In the segmentation in step 2.6.2, the class with the highest probability is assigned to each pixel. To apply hysteresis thresholding to the probability map of a certain class, probabilities need to be calculated first.

In [17]:
probabilities = classifier.predict_proba(X)

The code below adds the probability maps to the napari viewer (for the two classes 'background' and 'archaeological structures' as described in section 2.6.1).

In [18]:
# Probability map for the first class ('background')
probabilitymap = probabilities[:,0].reshape(image.shape)
viewer.add_image(probabilitymap, opacity = 1.0, name = "Probability (class 1)")

# Probability map for the second class ('archaeological structures')
probabilitymap = probabilities[:,1].reshape(image.shape)
viewer.add_image(probabilitymap, opacity = 1.0, name = "Probability (class 2)")

<Image layer 'Probability (class 2)' at 0x2751b8f7520>

### 2.8.2. Hysteresis thresholding
For the application of hysteresis thresholding, there are two thresholds. Pixels that belong to the 'archaeological structures' class with a probability above the low threshold are only selected if they are connected to pixels above a higher, more stringent probability threshold (the high threshold). Both thresholds can be set in the code below.

In [19]:
reshaped_probabilities = probabilities[:,1].reshape(image.shape)
smoothed_probabilities = vigra.filters.gaussianSmoothing(reshaped_probabilities, 0.5)
low = 0.5     # Enter the low threshold
high = 0.6   # Enter the high threshold
hyst = filters.apply_hysteresis_threshold(smoothed_probabilities, low, high)
viewer.add_labels(hyst, name = "Segmentation after hysteresis thresholding (Class 2)")

<Labels layer 'Segmentation after hysteresis thresholding (Class 2)' at 0x2751bf2ee20>

### 2.8.3. Creation of individual objects
Regions are labelled so that they can be used for subsequent object classification. Pixels receive the same label if they are neighbours in horizontal, vertical or diagonal direction. Connectivity can be changed to '1' if regions with the same label should only consist of pixels connected in horizontal or vertical (and not in diagonal) direction.

In [20]:
labeled_hyst = label(hyst, connectivity = 2, background = 0)
viewer.add_labels(labeled_hyst, name = "Objects (Class 2)")

<Labels layer 'Objects (Class 2)' at 0x2751d18fb80>

# 3. Object classification
## 3.1. Import .h5 file with segmented objects
If the pixel classification has been done in another software package (e.g. ilastik), please import the H5 file with the segmentation by running the code below. Please adjust the file name in the first line if necessary. The file should be in the same folder as the Jupyter Notebook, or its path should be specified in the code below. 

In [102]:
filename = "Binary_image.h5"
with h5py.File(filename, "r") as f:    
    key = list(f.keys())[0]     
    if nrofdim == 2:
        labeled_hyst = f[key][:,:,0]
    else:
        labeled_hyst = f[key][:,:,:,0]      
    viewer.add_labels(labeled_hyst, name = "Segmentation")

In [109]:
labeled_hyst.dtype

dtype('uint32')

## 3.2. Watershed segmentation

Often, the objects are too big for straightforward classification, because they encompass pixels belonging to more than one semantic class (for example parts of a wall and a floor). Watershed is applied to reduce the size of the regions and make them more semantically homogeneous. The size of the resulting regions can be controlled by the size of the footprint in the code below. If no watershed is to be used, please go immediately to the last cell of this section.<br>
The code is based on: https://scikit-image.org/docs/stable/auto_examples/segmentation/plot_watershed.html

In [21]:
labeled_hyst = labeled_hyst.astype(np.int32)
distance = ndi.distance_transform_edt(labeled_hyst.astype(np.int32))
if nrofdim == 2:    
    coords = peak_local_max(distance, footprint=np.ones((30,30)), labels=labeled_hyst)     
else:
    coords = peak_local_max(distance, footprint=np.ones((20,20,20)), labels=labeled_hyst)
mask = np.zeros(distance.shape, dtype=bool)
mask[tuple(coords.T)] = True
markers, _ = ndi.label(mask)
labeled_hyst_watershed = watershed(-distance, markers, mask=labeled_hyst)
viewer.add_labels(labeled_hyst_watershed, opacity = 0.8)

<Labels layer 'labeled_hyst_watershed' at 0x2751bc5ca90>

If no watershed is used, please run the code below:

In [7]:
labeled_hyst_watershed = labeled_hyst

## 3.3. Machine learning based object classification
### 3.3.1. Selection of object features
First run the code below to calculate region properties and create the object feature stack.

In [22]:
stats = regionprops(labeled_hyst_watershed, intensity_image=image)
object_feature_stack = []

A large number of object features exist, see the list in https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops.<br> Not all features are supported for 3D data. If you are analysing 3D data, please run the code below for lists of supported and unsupported 3-D features (the code is based on: https://scikit-image.org/skimage-tutorials/lectures/three_dimensional_image_processing.html)

In [23]:
supported = [] 
unsupported = []

for s in stats[0]:
    try:
        stats[0][s]
        supported.append(s)
    except NotImplementedError:
        unsupported.append(s)

print("Supported properties:")
print("  " + "\n  ".join(supported))
print()
print("Unsupported properties:")
print("  " + "\n  ".join(unsupported))

Supported properties:
  area
  area_bbox
  area_convex
  area_filled
  axis_major_length
  axis_minor_length
  bbox
  centroid
  centroid_local
  centroid_weighted
  centroid_weighted_local
  coords
  eccentricity
  equivalent_diameter_area
  euler_number
  extent
  feret_diameter_max
  image
  image_convex
  image_filled
  image_intensity
  inertia_tensor
  inertia_tensor_eigvals
  intensity_max
  intensity_mean
  intensity_min
  label
  moments
  moments_central
  moments_hu
  moments_normalized
  moments_weighted
  moments_weighted_central
  moments_weighted_hu
  moments_weighted_normalized
  orientation
  perimeter
  perimeter_crofton
  slice
  solidity

Unsupported properties:
  


C:\Users\Lieven\miniconda3\envs\envLV09\lib\site-packages\skimage\measure\_moments.py:326: RuntimeWarning: invalid value encountered in scalar power
  nu[powers] = (mu[powers] / scale ** sum(powers)) / (mu0 ** (sum(powers) / nu.ndim + 1))


A few combinations of object features are given below (for GPR and for magnetometer). Object features can be appended to the feature stack by adding a code line of the same form and including the name of the feature from the list on https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops.

In [136]:
# GPR data
#object_feature_stack.append(np.asarray([s.eccentricity for s in stats]))
#object_feature_stack.append(np.asarray([s.orientation for s in stats]))
object_feature_stack.append(np.asarray([s.solidity for s in stats]))

C:\Users\Lieven\miniconda3\envs\envLV09\lib\site-packages\skimage\measure\_regionprops.py:433: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 792261220  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width  9  Error-roundoff 8.1e-15  _one-merge 5.6e-14
  _near-inside 2.8e-13  Visible-distance 1.6e-14  U-max-coplanar 1.6e-14
  Width-outside 3.2e-14  _wide-facet 9.7e-14  _maxoutside 6.4e-14

  return convex_hull_image(self.image)
C:\Users\Lieven\miniconda3\envs\envLV09\lib\site-packages\skimage\measure\_regionprops.py:632: RuntimeWarning: divide by zero encountered in scalar divide
  return self.area / self.area_convex
C:\Users\Lieven\miniconda3\envs\envLV09\lib\site-packages\skimage\measure\_regionprops.py:433: UserWarning: Failed to ge

In [24]:
# Magnetometer data
object_feature_stack.append(np.asarray([s.area for s in stats]))
object_feature_stack.append(np.asarray([s.eccentricity for s in stats]))
object_feature_stack.append(np.asarray([s.solidity for s in stats]))

In [25]:
#The following code can be used to classify small dipoles in magnetometer data
areas = np.asarray([s.area for s in stats])
intensity_min = np.asarray([s.intensity_min for s in stats])
intensity_max = np.asarray([s.intensity_max for s in stats])
intensity_diff = intensity_max - intensity_min
ratio_intensity_size = intensity_diff / areas
object_feature_stack.append(ratio_intensity_size)

In [26]:
object_features = np.asarray(object_feature_stack)
Xobj = object_features.T
Xobj[Xobj == np.inf] = 0

## 3.3.2. Load classifier
If an object classifier has been previously trained and saved, it can be loaded here. Please specify its name in the code below. It should be in the same folder as the Jupyter Notebook, or its path should be specified in the code below. After loading the classifier, please go immediately to step 3.3.3.4.

In [ ]:
f = open('Object_classifier.pckl', 'rb')
classifierObj = pickle.load(f)
f.close()

## 3.3.3. Interactive training and classification
If no classifier has been loaded, please first run the code below to add the 'training_labels_object' layer to napari. Afterwards, if you would like to use raster labels for the classification, please go to step 3.3.3.1; if you would like to use vector shapes, please go to step 3.3.3.2.

In [28]:
training_labels_object = np.zeros(image.shape[:3], dtype=np.uint8)
viewer.add_labels(training_labels_object)

<Labels layer 'training_labels_object' at 0x2752748a7c0>

### 3.3.3.1. Creating training data by means of raster labels
For the iterative training and classification, please carry out the procedure below.
1. Train the classifier by selecting the 'training_labels_object' layer in napari and adding labels to it, and do so for as many classes as needed, using the paint brush and other instruments (see the icons in the top left corner of the viewer under 'Layer controls'). To label an object, it is sufficient that at least one pixel belonging to the object is labeled. If an object has received different labels, the label with the highest class number prevails.
2. When the labels have been added, run the code below and then go to steps 3.3.3.3 and 3.3.3.4. 
3. After running the code in 3.3.3.3 and 3.3.3.4, additional labels can be added in napari (in the 'training_labels_object' layer), and classification can be carried out again by running the code below and in steps 3.3.3.3 and 3.3.3.4, in an iterative way, until the result of the classification is satisfactory.

In [32]:
training_labels_object = viewer.layers['training_labels_object'].data

### 3.3.3.2. Creating training data by means of vectors (polygons, lines etc.)
Please first run the code below to add a 'shapes' layer to napari. Also indicate the class to which the objects covered by the shapes should belong (second line in the code below). All the objects should belong to the same class. If you are analysing 3D data, please also indicate the layer number in which you will label the objects (the layer number can be seen in the bottom right corner of the napari viewer when working with 3D data).

In [35]:
viewer.add_shapes(name = 'shapes')
classnumber = 1
layer = 16

For the iterative training and classification using polygons, please carry out the procedure below.

1. Select the 'shapes' layer in napari and add shapes to it, using the 'Add polygons' icon or other instruments (see the icons in the top left corner of the viewer, under 'Layer controls'). To label an object, it is sufficient that at least one pixel belonging to the object is covered by the shape.
2. After having drawn the shapes, please run the code below, which converts the shapes to raster labels and adds them to the 'training_labels_object' layer.
3. Additional shapes can be added in napari, using the same workflow: first create a new shapes layer and indicate the class to which the objects should be assigned (code cell above), then draw the shapes, and run the code below.
4. Once all shapes have been added, please go to steps 3.3.3.3 and 3.3.3.4.
5. After running the code in 3.3.3.3 and 3.3.3.4, additional vector labels can be added in napari, and classification can be carried out again by running the code below and in steps 3.3.3.3 and 3.3.3.4, in an iterative way, until the result of the classification is satisfactory.

In [36]:
shapes = viewer.layers['shapes']
viewer.layers.remove('shapes')
if nrofdim == 2:
    labels = shapes.to_labels(image.shape)    
else:
    labels = np.zeros(image.shape,dtype=np.uint8)
    labels[layernumber,:,:] = shapes.to_labels(image.shape[1:3])
training_labels_object[labels > 0] = classnumber

### 3.3.3.3. Training the classifier
After the training data has been created in steps 3.3.3.1 and/or 3.3.3.2., please run the code below to train the classifier

In [37]:
if 'class_image' in viewer.layers: 
    viewer.layers.remove('class_image')
yobj = training_labels_object.ravel()
annotation_stats = regionprops(labeled_hyst_watershed, intensity_image=training_labels_object)
annotated_class = np.asarray([s.intensity_max for s in annotation_stats])
maskObj = annotated_class > 0
XobjSel = Xobj[maskObj]
annotated_class_sel = annotated_class[maskObj]
classifierObj = RandomForestClassifier(n_jobs=-1)
classifierObj.fit(XobjSel, annotated_class_sel)

RandomForestClassifier(n_jobs=-1)

### 3.3.3.4. Applying the classifier to the entire dataset
After the classification has been completed, the result of the classification is automatically added as the 'class_image' layer in napari. The labels can be saved by selecting the 'training_labels_object' layer and applying 'File -> Save Selected Layer(s)' in napari.
The classification can be saved by selecting the 'class_image' layer and applying 'File -> Save Selected Layer(s)'.

In [38]:
resultObj = classifierObj.predict(Xobj)
resultObj0 = [0] + resultObj.tolist()
numel = len(resultObj0)
class_image = np.zeros(image.shape[:3], dtype=np.uint8)
for k in range(numel):    
    class_image[labeled_hyst_watershed == k] = resultObj0[k]
viewer.add_labels(class_image, opacity = 0.8)

<Labels layer 'class_image' at 0x2751d19e3d0>

The following code is faster for 3D data.

In [151]:
resultObj = classifierObj.predict(Xobj)
numel = len(resultObj)
class_image = np.zeros(image.shape[:3], dtype=np.uint8)
for k in range(numel):  
    if resultObj[k] > 0:     
        startz = annotation_stats[k].bbox[0]
        stopz = annotation_stats[k].bbox[3]
        starty = annotation_stats[k].bbox[1]
        stopy = annotation_stats[k].bbox[4]
        startx = annotation_stats[k].bbox[2]
        stopx = annotation_stats[k].bbox[5]
        class_image_portion = class_image[startz:stopz,starty:stopy,startx:stopx]
        labeled_hyst_watershed_portion = labeled_hyst_watershed[startz:stopz,starty:stopy,startx:stopx]
        class_image_portion[labeled_hyst_watershed_portion == k + 1] = resultObj[k]
        class_image[startz:stopz,starty:stopy,startx:stopx] = class_image_portion
viewer.add_labels(class_image, opacity = 0.8)

<Labels layer 'class_image' at 0x2474b382460>

## 3.3.4. Save classifier
To save the classifier, please specify its name below. In the default code below, it is saved in the same folder as the Jupyter Notebook. Otherwise, its path should be specified. 

In [152]:
f = open('Object_classifier.pckl', 'wb')
pickle.dump(classifierObj, f)
f.close()

# 4. Manual classification

## 4.1. Manual assignment of all objects to one class
If the vast majority of the objects belong to one class, it may be more straightforward to assign them manually to this class than to perform machine learning based object classification (section 3.3). In that case, run the code below and first adjust the class number in the first line. The objects not belonging to this class can then be assigned to their correct class in steps 4.2, 4.3 and 4.4.

In [ ]:
classnumber = 1
class_image = np.zeros(image.shape[:3], dtype=np.uint8)
class_image[labeled_hyst_watershed > 0] = classnumber
viewer.add_labels(class_image, opacity = 0.8)

## 4.2. Manual classification of individual objects
Objects wrongly classified by the classifier in section 3.3, or manually in section 4.1, can be manually assigned to a different class. Please first run the code below to add the 'manual_labels_object' labels layer to napari. Afterwards, if you would like to use raster labels for the classification, please go to step 4.2.1; if you would like to use vector shapes, please go to step 4.2.2.

In [39]:
manual_labels_object = np.zeros(image.shape[:3], dtype=np.uint8)
viewer.add_labels(manual_labels_object)

<Labels layer 'manual_labels_object' at 0x2752fbc4f40>

### 4.2.1. By means of raster labels

For the manual object classification, please carry out the procedure below.
1. Select the 'manual_labels_object' layer in napari and add labels to it, using the paint brush and other instruments (see the icons in the top left corner of the viewer under 'Layer controls'). To label an object, it is sufficient that at least one pixel belonging to the object is labeled. If an object has received different labels, the label with the highest class number prevails.
2. After having labelled the objects, please run the code below, and go to step 4.2.3. 

In [40]:
manual_labels_object = viewer.layers['manual_labels_object'].data

### 4.2.2. By means of vectors (polygons, lines etc.)
Please first run the code below to add a 'shapes' layer to napari. Also indicate the class to which the objects covered by the shapes should belong (second line in the code below). All the objects should belong to the same class. When working with 3D data, please also indicate the horizontal slice number in which you label the objects (the slice number can be seen in the bottom right corner of the napari viewer when working with 3D data).

In [42]:
viewer.add_shapes(name = 'shapes')
classnumber = 1
slicenumber = 9

For the manual object classification using vector labels, please carry out the procedure below. 
1. Select the 'shapes' layer in napari and add shapes to it, using the 'Add polygons' or other instruments (see the icons in the top left corner of the viewer, under 'Layer controls'). To label an object, it is sufficient that at least one pixel belonging to the object is covered by the shape.
2. After having drawn the shapes, please run the code below, which converts the shapes to raster labels and adds them to the 'manual_labels_object' layer.
3. Additional shapes can be added in napari, using the same workflow: first create a new 'shapes' layer and indicate the class to which the objects should be assigned (code cell above), then draw the shapes, and run the code below. 
4. Once all labels have been added, please go to step 4.2.3.

In [43]:
shapes = viewer.layers['shapes']
viewer.layers.remove('shapes')
if nrofdim == 2:
    labels = shapes.to_labels(image.shape)
else:    
    labels = np.zeros(image.shape,dtype=np.uint8)
    labels[slicenumber,:,:] = shapes.to_labels(image.shape[1:3])
manual_labels_object[labels > 0] = classnumber

### 4.2.3. Applying the modifications made
The modifications made in step 4.2.1 or 4.2.2 are applied to the 'class_image' layer by running the code below.
The labels can be saved by selecting the 'manual_labels_object' layer and applying 'File -> Save Selected Layer(s)' in napari. The classification can be saved by selecting the 'class_image' layer and applying 'File -> Save Selected Layer(s)'.

In [44]:
if 'class_image' in viewer.layers: 
    viewer.layers.remove('class_image')    
annotation_stats_man = regionprops(labeled_hyst_watershed, intensity_image=manual_labels_object)
annotated_class_man = np.asarray([s.intensity_max for s in annotation_stats_man])
annotated_class_man0 = [0] + annotated_class_man.tolist()
numel = len(annotated_class_man0)
for k in range(numel):     
    if annotated_class_man0[k] > 0:        
        class_image[labeled_hyst_watershed == k] = annotated_class_man0[k]
viewer.add_labels(class_image, opacity = 0.7)

<Labels layer 'class_image' at 0x2751d4e68e0>

The following code is faster for 3D data:

In [170]:
def medianOfNonZeros (regionmask, manual_labels_object):
    manual_labels_object_mask = manual_labels_object[regionmask]
    if np.mean(manual_labels_object_mask) == 0:
        return 0
    else:
        return np.median(manual_labels_object_mask[np.nonzero(manual_labels_object_mask)])
annotation_stats_man = regionprops(labeled_hyst_watershed, intensity_image=manual_labels_object, extra_properties=(medianOfNonZeros,))
annotated_class_man = np.asarray([s.medianOfNonZeros for s in annotation_stats_man])
numel = len(annotated_class_man)
for k in range(numel):  
    if annotated_class_man[k] > 0:     
        startz = annotation_stats_man[k].bbox[0]
        stopz = annotation_stats_man[k].bbox[3]
        starty = annotation_stats_man[k].bbox[1]
        stopy = annotation_stats_man[k].bbox[4]
        startx = annotation_stats_man[k].bbox[2]
        stopx = annotation_stats_man[k].bbox[5]
        class_image_portion = class_image[startz:stopz,starty:stopy,startx:stopx]
        labeled_hyst_watershed_portion = labeled_hyst_watershed[startz:stopz,starty:stopy,startx:stopx]
        class_image_portion[labeled_hyst_watershed_portion == k + 1] = annotated_class_man[k]
        class_image[startz:stopz,starty:stopy,startx:stopx] = class_image_portion
viewer.add_labels(class_image, opacity = 0.8)

<Labels layer 'class_image [1]' at 0x2476a9b0b20>

## 4.3. Pixel-based correction of existing objects (no modification of background)
Besides the manual re-classification of entire objects (section 4.2), parts of objects can be assigned to a different class, by labelling pixels belonging to that object. Please first run the code below to add the 'manual_labels_pixel' layer to napari. Afterwards, if you would like to use raster labels for the classification, please go to step 4.3.1; if you would like to use vector shapes, please go to step 4.3.2.

In [45]:
manual_labels_pixel = np.zeros(image.shape[:3], dtype=np.uint8)
viewer.add_labels(manual_labels_pixel)

<Labels layer 'manual_labels_pixel' at 0x2752fa4fbe0>

### 4.3.1. By means of raster labels

For the manual pixel-based correction, please carry out the procedure below.
1. Add labels to the 'manual_labels_pixel' layer in napari, using the paint brush and other instruments. This can be done for all existing classes. Only the pixels that belong to an already existing object, will be re-classified. Where the label is painted outside an existing object (i.e. in the background), it remains background. 
2. When the labels have been added, please run the code below, and go to step 4.3.3.

In [46]:
manual_labels_pixel = viewer.layers['manual_labels_pixel'].data

In 3D data, the correction will only apply to the horizontal slice which was annotated. If you want to apply it to all slices, please run the code below. Indicate the slice which was annotated in the first line of the code.

In [176]:
slicenumber = 9
manual_labels_pixel_layer = manual_labels_pixel[slicenumber,:,:]
manual_labels_pixel = np.tile(manual_labels_pixel_layer,[image.shape[0],1,1])
viewer.add_labels(manual_labels_pixel)

<Labels layer 'manual_labels_pixel [1]' at 0x24541830460>

### 4.3.2. By means of vector polygons
Please first run the code below to add a 'shapes' layer to napari. Also indicate the class to which the pixels covered by the shapes should belong (second line in the code below). All re-classified pixels should belong to the same class. When working with 3D data, please also indicate the slice number in which you label the objects (the slice number can be seen in the bottom right corner of the napari viewer).

In [48]:
viewer.add_shapes(name = 'shapes')
classnumber = 1
slicenumber = 16

For the manual pixel-based classification using vector labels, please carry out the procedure below.

1. Select the 'shapes' layer in napari and add shapes to it, using the 'Add polygons' or other instruments (see the icons in the top left corner of the viewer, under 'Layer controls'). Only the pixels that belong to an already existing object, will be re-classified. Where the shape extends outside an existing object (i.e. in the background), the background remains unchanged.
2. After having drawn the shapes, please run the code below, which converts the shapes to raster labels and adds them to the 'manual_labels_pixel' layer.
3. In 3D data, the correction will only apply to the horizontal slice which was annotated if the variable 'applyToAllSlices' is set to 'N' (first line of the code below). If you want the correction to apply to all slices, please set this variable to 'Y'.
4. Additional shapes can be added in napari, using the same workflow: first create a new shapes layer and indicate the class to which the objects should be assigned (code cell above), then draw the shapes, and run the code below.
5. Once all labels have been added, please go to step 4.3.3.

In [49]:
applyToAllSlices = 'Y' # For 3D data: apply changes only to slice which was annotated ('N') or to all slices ('Y')
shapes = viewer.layers['shapes']
viewer.layers.remove('shapes')
if nrofdim == 2:
    labels = shapes.to_labels(image.shape)
else:
    labels = np.zeros(image.shape,dtype=np.uint8)
    labels_slice = shapes.to_labels(image.shape[1:3])
    if applyToAllSlices == 'N':
        labels[slicenumber,:,:] = labels_slice
    else:
        labels = np.tile(labels_slice,[image.shape[0],1,1])
manual_labels_pixel[labels > 0] = classnumber

### 4.3.3. Applying the modifications made
The modifications made in step 4.3.1 or 4.3.2 are applied to the 'class_image' layer by running the code below. The labels can be saved by selecting the 'manual_labels_pixel' layer and applying 'File -> Save Selected Layer(s)' in napari. The classification can be saved by selecting the 'class_image' layer and applying 'File -> Save Selected Layer(s)'.

In [50]:
if 'class_image' in viewer.layers: 
    viewer.layers.remove('class_image')
classMax = np.max(manual_labels_pixel)
for k in range(classMax):
    class_image[manual_labels_pixel == k + 1] = k + 1    
class_image[labeled_hyst_watershed == 0] = 0
viewer.add_labels(class_image, opacity = 0.65)

<Labels layer 'class_image' at 0x27530ac6430>

## 4.4. Pixel-based correction (including background), or creation of new objects
Whereas in section 4.3 pixels can be re-classified only if they belong to an existing object, in this section any pixel can be re-classified (including the background) by labelling it. In this way also new objects can be created. 
1. If you would like to use raster labels for this correction, simply add labels to the 'class_image' layer, using the paint brush and other instruments. Where the label is painted outside existing objects (i.e. in the background class), these pixels are assigned to the new class. It may be useful to save the 'class_image' layer first, by selecting this layer and applying 'File -> Save Selected Layer(s)' in napari (because changes can only be undone manually with the eraser and paint brush).
2. If you would like to use vector shapes for the correction, please first run the code below to add a 'shapes' layer to napari. Also indicate the class to which the pixels covered by the shapes should belong (second line in the code below). All re-classified pixels should belong to the same class. When working with 3D data, please also indicate the horziontal slice number in which you label the objects (the slice number can be seen in the bottom right corner of the napari viewer).

In [51]:
viewer.add_shapes(name = 'shapes')
classnumber = 3
slicenumber = 6

For the manual pixel-based correction (including background), please carry out the procedure below.

1. Select the 'shapes' layer in napari and add shapes to it, using the 'Add polygons' or other instruments (see the icons in the top left corner of the viewer, under 'Layer controls'). Where the shape is drawn outside existing objects (i.e. in the background class), these pixels are assigned to the new class. In this way, this tool can be used for the creation of new objects.
2. After having drawn the shapes, please run the code below, which converts the shapes to raster labels and adds them to the 'class_image' layer.
3. Additional shapes can be added in napari, using the same workflow: first create a new shapes layer and indicate the class to which the objects should be assigned (code cell above), then draw the shapes, and run the code below.

In [52]:
shapes = viewer.layers['shapes']
viewer.layers.remove('shapes')
if nrofdim == 2:
    labels = shapes.to_labels(image.shape)
else:    
    labels = np.zeros(image.shape,dtype=np.uint8)
    labels[slicenumber,:,:] = shapes.to_labels(image.shape[1:3])    
class_image[labels > 0] = classnumber

# 5. Postprocessing
## 5.1. Creation of 2-D image from 3-D data
When analysing 3D data, it may be useful to project the 3D segmentation onto a 2D map, e.g. for use in a GIS. To create a 2D map from a 3D label volume, please run the code below.

In [213]:
class_image_slice = np.zeros(image.shape[1:3], dtype=np.uint8)
for m in range(image.shape[1]):    
    for n in range(image.shape[2]):
        class_image_column = class_image[:,m,n]
        if np.mean(class_image_column) == 0:
            class_image_slice[m,n] = 0            
        else:
            class_image_slice[m,n] = np.median(class_image_column[np.nonzero(class_image_column)])             
viewer.add_labels(class_image_slice, opacity = 0.65)

<Labels layer 'class_image_slice' at 0x2475704c2b0>

## 5.2. Creation of separate labels layers for each class
A text file with the class names needs to be provided. In this file, the text must be of the form: {"1": "Ditch", "2": "Road", "3": "House"}. In the default code below, this text file is named 'Classes_test.txt'. If it is named differently, please specify its name below. It should be in the same folder as the Jupyter Notebook, or otherwise its path should be specified in the code below.<br> Within the folder where the input data is located, a 'Classes' folder will be created (first code cell below). In that folder, for each class a file with the objects belonging to that class will be stored (second code cell below). When working with 3D data, please indicate in the first line of the second code cell below if the files representing individual classes should be 2D (i.e. extracted from the 2D map created in step 5.1) or 3D.

In [63]:
#classImageMax = int(np.max(class_image))
with open("Classes_test.txt","r") as file:
    classes = json.loads(file.read())
pathname = os.path.dirname(viewer.layers[0].source.path)
pathnameClasses = pathname + '/Classes/'
pathnameClassesExist = os.path.exists(pathnameClasses)
if not pathnameClassesExist:
    os.makedirs(pathnameClasses)   

In [54]:
filesIn3D = 'Y' # For 3D data: make 2D layers representing individual classes ('N') or 3D layers ('Y') 
for m in range(len(classes)):
    classnr = int(list(classes)[m])
    if nrofdim == 2:
        objectclass = np.zeros(image.shape, dtype=np.uint8)
        objectclass[class_image == classnr] = classnr
    else:
        if filesIn3D == 'N':
            objectclass = np.zeros(image.shape[1:3], dtype=np.uint8)
            objectclass[class_image_slice == classnr] = classnr
        else:
            objectclass = np.zeros(image.shape, dtype=np.uint8)
            objectclass[class_image == classnr] = classnr
    mstr = str(classnr)
    classname = classes[mstr]
    viewer.add_labels(objectclass, opacity = 0.8, name = classname)
    fullname = pathnameClasses + classname + '.tif'
    viewer.layers[classname].save(fullname)  

## 5.3. Morhological operations
With the code below, morphological operations can be performed. First, indicate the number of the class to which the operation should be applied and the radius of the structuring element. Other structuring elements can be used, see https://scikit-image.org/docs/stable/api/skimage.morphology.html. 

In [60]:
classnumber = 3
classnrstr = str(classnumber)
classname = classes[classnrstr]
original = viewer.layers[classname].data
radius = 3
if nrofdim == 2:
    footprint = morphology.disk(radius)
else:
    footprint = morphology.ball(radius)

Four operations can be performed: morphological dilation, erosion, closing and opening. For other operations, see https://scikit-image.org/docs/stable/api/skimage.morphology.html. The result of the operation is saved in the 'Classes' folder created in section 5.2.

In [193]:
# Morphological dilation
processed = morphology.dilation(original, footprint)
viewer.add_labels(processed, opacity = 0.8, name = classname + '_morph_dilation')
fullnameMorphDilat = pathnameClasses + classname + '_morph_dilation.tif'
viewer.layers[classname + '_morph_dilation'].save(fullnameMorphDilat)

['C:/Archeologie/Machine_learning/scikit-learn/Figuren_Interamna/3-D/7001-8000_6001-7000/Classes/Walls_morph_dilation.tif']

In [61]:
# Morphological erosion
processed = morphology.erosion(original, footprint)
viewer.add_labels(processed, opacity = 0.8, name = classname + '_morph_erosion')
fullnameMorphErosion = pathnameClasses + classname + '_morph_erosion.tif'
viewer.layers[classname + '_morph_erosion'].save(fullnameMorphErosion)

['C:/Archeologie/Machine_learning/scikit-learn/Figuren_Boviolles/Volledige_dataset/20241102_test_Jupyter_notebook/Classes/Linear_feature_morph_erosion.tif']

In [195]:
# Morphological opening
processed = morphology.opening(original, footprint)
viewer.add_labels(processed, opacity = 0.8, name = classname + '_morph_opening')
fullnameMorphOpen = pathnameClasses + classname + '_morph_opening.tif'
viewer.layers[classname + '_morph_opening'].save(fullnameMorphOpen)

['C:/Archeologie/Machine_learning/scikit-learn/Figuren_Interamna/3-D/7001-8000_6001-7000/Classes/Walls_morph_opening.tif']

In [196]:
# Morphological closing
processed = morphology.closing(original, footprint)
viewer.add_labels(processed, opacity = 0.8, name = classname + '_morph_closing')
fullnameMorphClos = pathnameClasses + classname + '_morph_closing.tif'
viewer.layers[classname + '_morph_closing'].save(fullnameMorphClos)

['C:/Archeologie/Machine_learning/scikit-learn/Figuren_Interamna/3-D/7001-8000_6001-7000/Classes/Walls_morph_closing.tif']

# 6. Convert raster images to polygons, and save them as shapefiles
Please provide the following information, by adapting the code below:
1. x-coordinate (Easting) and y-coordinate (Northing) of the lower left corner of the input data;
2. sample distance in x- and y-direction (in m);
3. coordinate reference system (EPSG code: https://epsg.org/home.html). <br>

If the input image is a GeoTIFF, this information is automatically extracted by running the code in the first cell below. Otherwise it should be entered manually in the second cell below.

In [62]:
# This code is based on: https://stackoverflow.com/questions/79059686/read-xyz-geotiff-file-with-tifffile
with tifffile.TiffFile(path) as tif:
    tif_tags = {}
    for tag in tif.pages[0].tags.values():
        name, value = tag.name, tag.value
        tif_tags[name] = value
coordx = tif_tags['ModelTiepointTag'][3]
coordy = tif_tags['ModelTiepointTag'][4] - tif_tags['ImageLength'] * tif_tags['ModelPixelScaleTag'][1]
sampledistx = tif_tags['ModelPixelScaleTag'][0]
sampledisty = tif_tags['ModelPixelScaleTag'][1]
CoordRefSyst = 'EPSG:' + str(tif_tags['GeoKeyDirectoryTag'][15])

In [197]:
coordx = 395750 #395600 #876899 #876649
coordy = 4586831 #4586781 #6840225 #6839975
sampledistx = 0.05 #0.25
sampledisty = 0.05 #0.25
CoordRefSyst = "EPSG:32633" #"EPSG:32631"

To run the code below, a text file mentioning the classes should be provided. In this file, the text should be of the form: {"1": "Ditch", "2": "Road", "3": "House"}. In the default code below, this text file is named 'Classes_test.txt'. If it is named differently, please specify its name below. It should be in the same folder as the Jupyter Notebook, or otherwise its path should be specified in the code below. <br> A subfolder called 'Shapefiles' will be created in the folder where input data is located. For each class, a shapefile with the name of the class is created. The path and file names can be changed by modifying the 'pathname' and 'fullname' variables in the code below.

In [68]:
with open("Classes_Boviolles2.txt","r") as file:
    classes = json.loads(file.read())
pathname = os.path.dirname(viewer.layers[0].source.path)
pathnameshp = pathname + '/Shapefiles/'
pathnameshpExist = os.path.exists(pathnameshp)
if not pathnameshpExist:
    os.makedirs(pathnameshp)

In [69]:
# This code is based on: 
# https://docs.opencv.org/3.4/d4/d73/tutorial_py_contours_begin.html
# https://docs.opencv.org/4.x/d9/d8b/tutorial_py_contours_hierarchy.html
# https://stackoverflow.com/questions/57965493/how-to-convert-numpy-arrays-obtained-from-cv2-findcontours-to-shapely-polygons
# https://gis.stackexchange.com/questions/353057/how-to-create-a-shapely-polygon-with-a-hole
# https://shapely.readthedocs.io/en/stable/reference/shapely.MultiPolygon.html
# https://shapely.readthedocs.io/en/2.0.3/manual.html

for r in range(len(classes)):
    classnr = list(classes)[r]   
    classname = classes[classnr]
    classbinarymap = viewer.layers[classname].data
    classbinarymap[classbinarymap == int(classnr)] = 1   
    classbinarymap2 = np.zeros(classbinarymap.shape, dtype=np.uint8)
    
    # Label connected components
    classbinarymaplabeled = label(classbinarymap, connectivity=1)
    
    # Compute region properties
    stats = regionprops(classbinarymaplabeled)
    
    # Filter bounding boxes
    for q, s in enumerate(stats):
        box = s.bbox
        if box[2] - box[0] > 1 and box[3] - box[1] > 1:
            classbinarymap2[classbinarymaplabeled == q + 1] = 1
    
    # Find contours
    #contours, _ = cv.findContours(classbinarymap2, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    contours, hierarchy = cv.findContours(classbinarymap2, cv.RETR_CCOMP, cv.CHAIN_APPROX_NONE)
    #contoursSq = map(np.squeeze, contours)

    # Convert contours to polygons. Take into account possible holes in polygons using the hierarchy of the contours
    listofpolygons = []
    for k in range(len(contours)):
    	if hierarchy[0][k][3] == -1:
    		exterior = []
    		holes = []
    		for m in range(len(contours[k])):
    			tuple_element = (contours[k][m][0][0],contours[k][m][0][1])
    			exterior.append(tuple_element)
    		for p in range(len(contours)):
    			if hierarchy[0][p][3] == k:
    				interior = []
    				for n in range(len(contours[p])):
    					tuple_element = (contours[p][n][0][0],contours[p][n][0][1])
    					interior.append(tuple_element)
    					interior[::-1]
    				holes.append(interior)
    		pgn = Polygon(exterior,holes)
    		listofpolygons.append(pgn)
    multipolygon = MultiPolygon(listofpolygons)
       
    # Apply geometric transformations
    multipolygonflipud1 = affine_transform(multipolygon,[1, 0, 0, -1, 0, 0])
    if nrofdim == 2:
        multipolygonflipud2 = translate(multipolygonflipud1, yoff=image.shape[0])
    elif nrofdim == 3:
        multipolygonflipud2 = translate(multipolygonflipud1, yoff=image.shape[1])
    multipolygonscaled = scale(multipolygonflipud2, xfact=sampledistx, yfact=sampledisty, origin=(0,0))    
    multipolygontransl = translate(multipolygonscaled, xoff=coordx, yoff=coordy)
    
    # Create GeoDataFrame
    gdf = gpd.GeoDataFrame(index=[0], geometry=[multipolygontransl], crs=CoordRefSyst)
    
    # Explode multipolygon into individual geometries
    gdf_exploded = gdf.explode()
    
    # Save shapefile    
    fullname = pathnameshp + classname + '.shp'
    gdf_exploded.to_file(fullname)

C:\Users\Lieven\AppData\Local\Temp\ipykernel_23244\1927754597.py:58: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  gdf_exploded = gdf.explode()
C:\Users\Lieven\AppData\Local\Temp\ipykernel_23244\1927754597.py:58: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  gdf_exploded = gdf.explode()
